In [33]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [34]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

<IPython.core.display.Javascript object>

In [35]:
import torch

if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
DEVICE = torch.device(dev)

<IPython.core.display.Javascript object>

In [37]:
import pandas as pd

data = pd.read_csv("data/train.csv")

<IPython.core.display.Javascript object>

In [38]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(
    data,
    test_size=0.2,
    shuffle=True,
    random_state=0,
    stratify=data["language"],
)

<IPython.core.display.Javascript object>

In [39]:
train_data.groupby("language").count()

,text,label
language,,
Chinese,1277,1277
English,1269,1269
French,1270,1270
Italian,1226,1226
Portuguese,1277,1277
Spanish,1273,1273


<IPython.core.display.Javascript object>

In [40]:
lang_id_map = {
    "English": "en",
    "Spanish": "es",
    "Portuguese": "pt",
    "Italian": "it",
    "French": "fr",
    "Chinese": "zh",
}

translate_id_map = {"Hindi": "hi", "Korean": "ko", "Arabic": "ar", "Dutch": "nl"}

<IPython.core.display.Javascript object>

In [41]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(
    DEVICE
)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

<IPython.core.display.Javascript object>

In [42]:
lang_id_map.update(translate_id_map)

<IPython.core.display.Javascript object>

In [48]:
import time


def get_text_translate(text, src_lang, to_lang):
    tokenizer.src_lang = lang_id_map[src_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(DEVICE)
    generated_tokens = model.generate(
        **encoded_text, forced_bos_token_id=tokenizer.get_lang_id(to_lang)
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


valid_translate = []
start_time = time.time()
for idx, row in valid_data.iterrows():
    for lang, lang_id in translate_id_map.items():
        to_translate = {"label": row["label"]}
        to_translate["language"] = row["language"]
        to_translate["text"] = row["text"]
        valid_translate.append(to_translate)
        translation = get_text_translate(row["text"], row["language"], lang_id)
        to_translate["language"] = lang
        to_translate["text"] = translation
        valid_translate.append(to_translate)
    if idx % 100 == 0:
        print(f"Translated {idx} rows, time: {time.time() - start_time}")

/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated 5700 rows, time: 348.3814821243286
Translated 6800 rows, time: 1300.767899274826
Translated 1000 rows, time: 1897.9547016620636
Translated 3200 rows, time: 2418.0205433368683
Translated 7600 rows, time: 2731.2635521888733
Translated 4200 rows, time: 2958.6970460414886
Translated 1600 rows, time: 3011.8779911994934
Translated 3800 rows, time: 3019.2263674736023
Translated 6700 rows, time: 3330.1551554203033
Translated 7400 rows, time: 3337.9796421527863
Translated 2500 rows, time: 3350.240747690201
Translated 500 rows, time: 3441.387348651886
Translated 4400 rows, time: 3540.769341945648
Translated 2800 rows, time: 3677.340935945511
Translated 3500 rows, time: 3966.8482072353363
Translated 8600 rows, time: 4161.120901823044
Translated 1800 rows, time: 4384.716200351715


<IPython.core.display.Javascript object>

In [49]:
valid_translate_df = pd.DataFrame(valid_translate)
valid_translate_df.to_csv("data/valid_translate.tsv", sep="\t")

<IPython.core.display.Javascript object>

In [46]:
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899 entries, 2309 to 8660
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      1899 non-null   object 
 1   label     1899 non-null   float64
 2   language  1899 non-null   object 
dtypes: float64(1), object(2)
memory usage: 59.3+ KB


<IPython.core.display.Javascript object>

In [50]:
valid_translate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15192 entries, 0 to 15191
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   label     15192 non-null  float64
 1   language  15192 non-null  object 
 2   text      15192 non-null  object 
dtypes: float64(1), object(2)
memory usage: 356.2+ KB


<IPython.core.display.Javascript object>

In [12]:
import time

def get_text_translate(text, src_lang, to_lang):
    tokenizer.src_lang = lang_id_map[src_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(DEVICE)
    generated_tokens = model.generate(
        **encoded_text, forced_bos_token_id=tokenizer.get_lang_id(to_lang)
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


full_translate = []
start_time = time.time()
for idx, row in train_data.iterrows():

    for lang, lang_id in lang_id_map.items():
        to_translate = {"label": row["label"]}
        to_translate["language"] = lang
        if lang == row["language"]:
            to_translate["text"] = row["text"]
            full_translate.append(to_translate)
            continue
        translation = get_text_translate(row["text"], row["language"], lang_id)
        to_translate["text"] = translation
        full_translate.append(to_translate)
    if idx % 100 == 0:
        print(f"Translated {idx} rows, time: {time.time() - start_time}")

/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated 2000 rows, time: 1055.413194179535
Translated 5100 rows, time: 2684.9987077713013
Translated 8800 rows, time: 2921.2999069690704
Translated 2100 rows, time: 3463.0867998600006
Translated 5400 rows, time: 3658.5993473529816
Translated 1400 rows, time: 4072.225198507309
Translated 6000 rows, time: 4074.7356581687927
Translated 5500 rows, time: 4119.256384849548
Translated 3100 rows, time: 4395.836714029312
Translated 4900 rows, time: 4552.5313675403595
Translated 9100 rows, time: 4630.211196184158
Translated 800 rows, time: 4672.650715351105
Translated 200 rows, time: 5886.027867078781
Translated 3300 rows, time: 5899.491555452347
Translated 100 rows, time: 6072.639766216278
Translated 8200 rows, time: 7168.2985055446625
Translated 3900 rows, time: 7354.317500829697
Translated 8400 rows, time: 7431.445020198822
Translated 6400 rows, time: 7511.999277830124
Translated 700 rows, time: 7702.463855266571
Translated 3700 rows, time: 7755.862390756607
Translated 0 rows, time: 8095.5

<IPython.core.display.Javascript object>

In [13]:
full_translate_df = pd.DataFrame(full_translate)
full_translate_df.to_csv("full_translate_all.tsv", sep="\t")

<IPython.core.display.Javascript object>

In [29]:
full_translate_df = pd.read_csv("full_translate_all.tsv", sep="\t")
del full_translate_df["Unnamed: 0"]

<IPython.core.display.Javascript object>

In [30]:
full_translate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75920 entries, 0 to 75919
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   label     75920 non-null  float64
 1   language  75920 non-null  object 
 2   text      75920 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.7+ MB


<IPython.core.display.Javascript object>

In [31]:
full_translate_df["text"] = full_translate_df["text"].apply(lambda x: x.strip("[]"))
full_translate_df.tail()

,label,language,text
75915,2.4,Chinese,"'我看到了你們的照片,用過濾器改變的孩子,給你們一個安靜的上帝。'"
75916,2.4,Hindi,'मैंने उन बच्चों की तस्वीरें देखी हैं जिन्हें ...
75917,2.4,Korean,"'나는 필터로 수정 된 아이들의 사진을 보았는데, 하느님께 진정으로 주세요.'"
75918,2.4,Arabic,'لقد رأيت صوركم من الأطفال المعدلين مع الفلاتر...
75919,2.4,Dutch,"""Ik heb foto's van jullie van kinderen gemodif..."


<IPython.core.display.Javascript object>

In [32]:
full_translate_df.to_csv("full_translate_all.tsv", sep="\t")

<IPython.core.display.Javascript object>

In [37]:
sampled_data = data.sample(len(data) // 6, random_state=3)

<IPython.core.display.Javascript object>

In [38]:
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1581 entries, 7033 to 4403
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      1581 non-null   object 
 1   label     1581 non-null   float64
 2   language  1581 non-null   object 
dtypes: float64(1), object(2)
memory usage: 49.4+ KB


<IPython.core.display.Javascript object>

In [39]:
# translate Hindi to French


def get_translate(x, to_lang):
    tokenizer.src_lang = lang_id_map[x["language"]]
    encoded_text = tokenizer(x["text"], return_tensors="pt").to(DEVICE)
    generated_tokens = model.generate(
        **encoded_text, forced_bos_token_id=tokenizer.get_lang_id(to_lang)
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


for lang, lang_id in translate_id_map.items():
    sampled_data[lang] = sampled_data.apply(lambda x: get_translate(x, lang_id), axis=1)

/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/soe/sgarg6/conda/envs/nlp_env/lib/python3.10/site-packages/transformers/generatio

<IPython.core.display.Javascript object>

In [40]:
sampled_data.head()

,text,label,language,Hindi,Korean,Arabic,Dutch
7033,Vient de publier une vidéo à Dubaï Émirat http,1.0,French,[हाल ही में एक वीडियो प्रकाशित किया गया है दुब...,[방금 두바이 에미리트 http에 비디오를 게시],[تم نشر مقطع فيديو في دبي الإمارات http],[Net een video gepubliceerd in Dubai Emiraten ...
6452,@user Avec d'autres passe temps,2.0,French,[दूसरों के साथ समय बिताने के लिए],[다른 사람들과 함께 시간을 보내고],[@المستخدم مع الآخرين يستغرق وقتا],[Gebruik met anderen tijd]
4599,"nossa to com a barriga tão grande, acho q vou ...",4.0,Portuguese,"[मेरे पेट इतना बड़ा है, मुझे लगता है कि मैं एक...","[우리 토 너무 큰 뱃속, 생각 q 나는 공장 p 그 슬픔 ]",[لدينا التوت مع البطن كبير جدا، أعتقد q سأطلب ...,"[Onze to met de buik zo groot, ik denk dat ik ..."
13,Dm to buy full vids of local coloured mense fu...,4.4,English,[Dm खरीदने के लिए पूर्ण vids के स्थानीय रंगीन ...,[Dm 구매 완전한 vids 로컬 염색 엿<unk> http],[Dm لشراء كاملة vids من المحلية ملونة mense ال...,[Dm om te kopen volledige vids van lokale gekl...
115,@user @user You have just said your grandchild...,1.6,English,[@user @user आपने बस कहा है कि आपका भतीजा ठंडा...,[@user @user 당신은 단지 당신의 조카가 차가운 것을 말했는데 어떻게 그가...,[@user @user قلت للتو أن حفيدك كان باردًا كيف ...,[@user @user Jij hebt net gezegd dat je kleink...


<IPython.core.display.Javascript object>

In [53]:
sampled_data.to_csv("sample_translated.csv", sep="\t")

<IPython.core.display.Javascript object>

In [44]:
sampled_data.iloc[0].to_dict()

{'text': 'Vient de publier une vidéo à Dubaï Émirat http',
 'label': 1.0,
 'language': 'French',
 'Hindi': ['हाल ही में एक वीडियो प्रकाशित किया गया है दुबई एमिरैट http'],
 'Korean': ['방금 두바이 에미리트 http에 비디오를 게시'],
 'Arabic': ['تم نشر مقطع فيديو في دبي الإمارات http'],
 'Dutch': ['Net een video gepubliceerd in Dubai Emiraten HTTP']}

<IPython.core.display.Javascript object>

In [49]:
translated_data = []

for idx, row in sampled_data.iterrows():
    row = row.to_dict()
    del row["text"], row["language"]
    for lang in translate_id_map.keys():
        to_append = {}
        to_append["label"] = row["label"]
        to_append["text"] = row[lang]
        to_append["language"] = lang
        translated_data.append(to_append)

<IPython.core.display.Javascript object>

In [50]:
translated = pd.DataFrame(translated_data)

<IPython.core.display.Javascript object>

In [51]:
translated.head()

,label,text,language
0,1.0,[हाल ही में एक वीडियो प्रकाशित किया गया है दुब...,Hindi
1,1.0,[방금 두바이 에미리트 http에 비디오를 게시],Korean
2,1.0,[تم نشر مقطع فيديو في دبي الإمارات http],Arabic
3,1.0,[Net een video gepubliceerd in Dubai Emiraten ...,Dutch
4,2.0,[दूसरों के साथ समय बिताने के लिए],Hindi


<IPython.core.display.Javascript object>

In [54]:
translated.to_csv("translated.tsv", sep="\t")

<IPython.core.display.Javascript object>

In [52]:

translated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6324 entries, 0 to 6323
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   label     6324 non-null   float64
 1   text      6324 non-null   object 
 2   language  6324 non-null   object 
dtypes: float64(1), object(2)
memory usage: 148.3+ KB


<IPython.core.display.Javascript object>